# Command-line staramr tutorial

This tutorial will walk you through running `staramr` on some example genomes to investigate AMR genes and point mutations.  The data we will use are two RefSeq assemblies that are available on NCBI: [GCF_001478105.1](https://www.ncbi.nlm.nih.gov/assembly/GCA_001478105.1) and [GCF_001931595.1](https://www.ncbi.nlm.nih.gov/assembly/GCA_001931595.1).

## Step 1: Download input files

Start by creating a folder where we will download the files and run `staramr`:

In [1]:
mkdir staramr_tutorial

Head to the folder by using the following command:

In [2]:
cd staramr_tutorial

You may download the input files with the following commands:

In [3]:
wget -O GCF_001478105.1.fasta.gz ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/001/478/105/GCF_001478105.1_Salmonella_enterica_CVM_N31384-SQ_v1.0/GCF_001478105.1_Salmonella_enterica_CVM_N31384-SQ_v1.0_genomic.fna.gz
gunzip GCF_001478105.1.fasta.gz

wget -O GCF_001931595.1.fasta.gz ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/001/931/595/GCF_001931595.1_ASM193159v1/GCF_001931595.1_ASM193159v1_genomic.fna.gz
gunzip GCF_001931595.1.fasta.gz

--2019-03-26 15:38:02--  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/001/478/105/GCF_001478105.1_Salmonella_enterica_CVM_N31384-SQ_v1.0/GCF_001478105.1_Salmonella_enterica_CVM_N31384-SQ_v1.0_genomic.fna.gz
           => ‘GCF_001478105.1.fasta.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 2607:f220:41e:250::12
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/all/GCF/001/478/105/GCF_001478105.1_Salmonella_enterica_CVM_N31384-SQ_v1.0 ... done.
==> SIZE GCF_001478105.1_Salmonella_enterica_CVM_N31384-SQ_v1.0_genomic.fna.gz ... 1454519
==> PASV ... done.    ==> RETR GCF_001478105.1_Salmonella_enterica_CVM_N31384-SQ_v1.0_genomic.fna.gz ... done.
Length: 1454519 (1.4M) (unauthoritative)

GCF_001478105.1_Sal 100%[===================>]   1.39M  5.58MB/s    in 0.2s    

2019-03-26 15:38:03 (5.58 MB/s) - ‘GCF_0

Now we have some files to work with:

In [4]:
ls

GCF_001478105.1.fasta  GCF_001931595.1.fasta


## Step 2: Run staramr

Now that we have some assembled genomes to work with, let's run `staramr`.

First, lets check what version of databases we are using:

In [5]:
staramr db info

2019-03-26 15:38:08,515 WARNING: Using non-default ResFinder/PointFinder/PlasmidFinder database versions. This may lead to differences in the detected AMR genes depending on how the database files are structured.
resfinder_db_dir              = /home/CSCScience.ca/jtran/Projects/staramr/staramr/databases/data/update/resfinder
resfinder_db_url              = https://bitbucket.org/genomicepidemiology/resfinder_db.git
resfinder_db_commit           = e8f1eb2585cd9610c4034a54ce7fc4f93aa95535
resfinder_db_date             = Mon, 16 Jul 2018 16:58
pointfinder_db_dir            = /home/CSCScience.ca/jtran/Projects/staramr/staramr/databases/data/update/pointfinder
pointfinder_db_url            = https://bitbucket.org/genomicepidemiology/pointfinder_db.git
pointfinder_db_commit         = 8706a6363bb29e47e0e398c53043b037c24b99a7
pointfinder_db_date           = Wed, 04 Jul 2018 14:27
plasmidfinder_db_dir          = /home/CSCScience.ca/jtran/Projects/staramr/staramr/databases/data/update/plasmidfin

Everything looks good there. Now, let's run `staramr`:

In [6]:
staramr search --pointfinder-organism salmonella -o out *.fasta

2019-03-26 15:38:10,508 WARNING: Using non-default ResFinder/PointFinder. This may lead to differences in the detected AMR genes depending on how the database files are structured.
2019-03-26 15:38:10,520 INFO: No --plasmidfinder-database-type specified. Will search the entire PlasmidFinder database
2019-03-26 15:38:10,520 INFO: --output-dir set. All files will be output to [out]
2019-03-26 15:38:10,521 INFO: Will exclude ResFinder/PointFinder genes listed in [/home/CSCScience.ca/jtran/Projects/staramr/staramr/databases/exclude/data/genes_to_exclude.tsv]. Use --no-exclude-genes to disable
2019-03-26 15:38:10,532 INFO: Making BLAST databases for input files
2019-03-26 15:38:10,609 INFO: Scheduling blasts for GCF_001478105.1.fasta
2019-03-26 15:38:10,622 INFO: Scheduling blasts for GCF_001931595.1.fasta
2019-03-26 15:38:12,854 INFO: Finished. Took 0.04 minutes.
2019-03-26 15:38:12,886 INFO: Predicting AMR resistance phenotypes is enabled. The predictions are for microbiological resistanc

There, that wasn't too long.

## Step 3: Examine results

Now, let's inspect some of the results. First, let's look at what files were produced:

In [7]:
ls out/

detailed_summary.tsv  plasmidfinder.tsv  resfinder.tsv  settings.txt
hits                  pointfinder.tsv    results.xlsx   summary.tsv


The __*.tsv__ files contain the primary results we're interested in. The **settings.txt** file contains all the settings used to run `staramr`. The **results.xlsx** file contains all the previous files as separate worksheets in an Excel file. And the **hits/** directory contains the AMR gene hits as FASTA files.

Let's take a look at these files in turn.

_Note that the command `column -s$'\t' -t file.tsv` is used. This command aligns the columns and prints a table `-t` using a tab character as the delimiter `-s$'\t'`._

In [8]:
column -s$'\t' -t out/summary.tsv

Isolate ID       Genotype                                                                                 Predicted Phenotype                                                                                                                                        Plasmid Genes
GCF_001478105.1  blaCMY-2                                                                                 ampicillin, amoxicillin/clavulanic acid, cefoxitin, ceftriaxone                                                                                            IncI1, IncX1
GCF_001931595.1  aac(3)-IVa, aph(3')-Ia, aph(4)-Ia, blaCTX-M-65, dfrA14, floR, gyrA (D87Y), sul1, tet(A)  gentamicin, kanamycin, hygromicin, ampicillin, ceftriaxone, trimethoprim, chloramphenicol, ciprofloxacin I/R, nalidixic acid, sulfisoxazole, tetracycline  None


Let's try and split up the table a bit to display better in GitHub.

In [9]:
# Show only Genotype
cut -f 1,2 out/summary.tsv | column -s$'\t' -t

Isolate ID       Genotype
GCF_001478105.1  blaCMY-2
GCF_001931595.1  aac(3)-IVa, aph(3')-Ia, aph(4)-Ia, blaCTX-M-65, dfrA14, floR, gyrA (D87Y), sul1, tet(A)


In [10]:
# Show only Predicted Phenotype
cut -f 1,3 out/summary.tsv | column -s$'\t' -t

Isolate ID       Predicted Phenotype
GCF_001478105.1  ampicillin, amoxicillin/clavulanic acid, cefoxitin, ceftriaxone
GCF_001931595.1  gentamicin, kanamycin, hygromicin, ampicillin, ceftriaxone, trimethoprim, chloramphenicol, ciprofloxacin I/R, nalidixic acid, sulfisoxazole, tetracycline


In [11]:
# Show only Plasmid Genes
cut -f 1,4 out/summary.tsv | column -s$'\t' -t

Isolate ID       Plasmid Genes
GCF_001478105.1  IncI1, IncX1
GCF_001931595.1  None


This file **summary.tsv** contains a summary of all the results in a single table, one genome per line. According to these results, the genomes *GCF_001478105.1* and *GCF_001931595.1* contain the listed AMR genes under **Genotype** and are resistant to the listed drugs under **Predicted Phenotype**. This also includes Plasmid Genes that were found in the genomes as well.

In [12]:
# Show all columns
column -s$'\t' -t out/resfinder.tsv

Isolate ID       Gene         Predicted Phenotype                                              %Identity  %Overlap  HSP Length/Total Length  Contig                  Start   End     Accession
GCF_001478105.1  blaCMY-2     ampicillin, amoxicillin/clavulanic acid, cefoxitin, ceftriaxone  100.00     100.00    1146/1146                ref|NZ_JYVD01000056.1|  25020   26165   X91840
GCF_001931595.1  aac(3)-IVa   gentamicin                                                       99.87      100.00    786/786                  ref|NZ_CP016411.1|      292669  291885  X01385
GCF_001931595.1  aph(3')-Ia   kanamycin                                                        100.00     100.00    816/816                  ref|NZ_CP016411.1|      300747  301562  X62115
GCF_001931595.1  aph(4)-Ia    hygromicin                                                       100.00     100.00    1026/1026                ref|NZ_CP016411.1|      291664  290639  V01499
GCF_001931595.1  blaCTX-M-65  ampicillin, ceftriaxone    

In [13]:
# Show a subset of columns
cut -f 1,2,4,5,6,7 out/resfinder.tsv | column -s$'\t' -t

Isolate ID       Gene         %Identity  %Overlap  HSP Length/Total Length  Contig
GCF_001478105.1  blaCMY-2     100.00     100.00    1146/1146                ref|NZ_JYVD01000056.1|
GCF_001931595.1  aac(3)-IVa   99.87      100.00    786/786                  ref|NZ_CP016411.1|
GCF_001931595.1  aph(3')-Ia   100.00     100.00    816/816                  ref|NZ_CP016411.1|
GCF_001931595.1  aph(4)-Ia    100.00     100.00    1026/1026                ref|NZ_CP016411.1|
GCF_001931595.1  blaCTX-M-65  100.00     100.00    876/876                  ref|NZ_CP016411.1|
GCF_001931595.1  dfrA14       99.79      100.00    483/483                  ref|NZ_CP016411.1|
GCF_001931595.1  floR         98.19      99.92     1214/1215                ref|NZ_CP016411.1|
GCF_001931595.1  sul1         100.00     100.00    840/840                  ref|NZ_CP016411.1|
GCF_001931595.1  tet(A)       100.00     100.00    1200/1200                ref|NZ_CP016411.1|


This shows all the BLAST hits to the **ResFinder** database, each hit on a single line.

In [14]:
# Show all columns
column -s$'\t' -t out/pointfinder.tsv

Isolate ID       Gene         Predicted Phenotype                Type   Position  Mutation             %Identity  %Overlap  HSP Length/Total Length  Contig              Start    End
GCF_001931595.1  gyrA (D87Y)  ciprofloxacin I/R, nalidixic acid  codon  87        GAC -> TAC (D -> Y)  99.43      100.00    2637/2637                ref|NZ_CP016410.1|  1597907  1600543


In [15]:
# Show a subset of columns
cut -f 1,2,5,6,7,8,10 out/pointfinder.tsv | column -s$'\t' -t

Isolate ID       Gene         Position  Mutation             %Identity  %Overlap  Contig
GCF_001931595.1  gyrA (D87Y)  87        GAC -> TAC (D -> Y)  99.43      100.00    ref|NZ_CP016410.1|


This shows all the aquired point mutations and plasmids genes leading to antimicrobial resistance, one per line.

In [16]:
# Show all columns
column -s$'\t' -t out/plasmidfinder.tsv

Isolate ID       Gene   %Identity  %Overlap  HSP Length/Total Length  Contig                  Start  End    Accession
GCF_001478105.1  IncI1  100.00     100.00    142/142                  ref|NZ_JYVD01000056.1|  15896  16037  AP005147
GCF_001478105.1  IncX1  100.00     100.00    373/373                  ref|NZ_JYVD01000049.1|  2546   2174   CP001123


In [17]:
# Show a subset of columns
cut -f 1,2,4,5,6 out/plasmidfinder.tsv | column -s$'\t' -t

Isolate ID       Gene   %Overlap  HSP Length/Total Length  Contig
GCF_001478105.1  IncI1  100.00    142/142                  ref|NZ_JYVD01000056.1|
GCF_001478105.1  IncX1  100.00    373/373                  ref|NZ_JYVD01000049.1|


This shows all the plasmid genes detected in the genomes.

The **detailed_summary.tsv** file contains a summary of all the results in a single table, where all genes found were shown per line and where it was found from either ResFinder, PlasmidFinder, or PointFinder.

In [18]:
# Show all columns
column -s$'\t' -t -n out/detailed_summary.tsv

Isolate ID       Gene         Predicted Phenotype                                              %Identity          %Overlap           HSP Length/Total Length  Contig                  Start      End        Accession  Data Type
GCF_001478105.1  IncI1                                                                         100.0              100.0              142/142                  ref|NZ_JYVD01000056.1|  15896.0    16037.0    AP005147   Plasmid
GCF_001478105.1  IncX1                                                                         100.0              100.0              373/373                  ref|NZ_JYVD01000049.1|  2546.0     2174.0     CP001123   Plasmid
GCF_001478105.1  blaCMY-2     ampicillin, amoxicillin/clavulanic acid, cefoxitin, ceftriaxone  100.0              100.0              1146/1146                ref|NZ_JYVD01000056.1|  25020.0    26165.0    X91840     Resistance
GCF_001931595.1  None                                                                                  

In [19]:
# Show a subset of columns
cut -f 1,2,4,7,11 out/detailed_summary.tsv | column -s$'\t' -t -n

Isolate ID       Gene         %Identity          Contig                  Data Type
GCF_001478105.1  IncI1        100.0              ref|NZ_JYVD01000056.1|  Plasmid
GCF_001478105.1  IncX1        100.0              ref|NZ_JYVD01000049.1|  Plasmid
GCF_001478105.1  blaCMY-2     100.0              ref|NZ_JYVD01000056.1|  Resistance
GCF_001931595.1  None                                                    Plasmid
GCF_001931595.1  aac(3)-IVa   99.87299999999999  ref|NZ_CP016411.1|      Resistance
GCF_001931595.1  aph(3')-Ia   100.0              ref|NZ_CP016411.1|      Resistance
GCF_001931595.1  aph(4)-Ia    100.0              ref|NZ_CP016411.1|      Resistance
GCF_001931595.1  blaCTX-M-65  100.0              ref|NZ_CP016411.1|      Resistance
GCF_001931595.1  dfrA14       99.79299999999999  ref|NZ_CP016411.1|      Resistance
GCF_001931595.1  floR         98.18799999999999  ref|NZ_CP016411.1|      Resistance
GCF_001931595.1  gyrA (D87Y)  99.431             ref|NZ_CP016410.1|      Resistance
GC

In [20]:
cat out/settings.txt

command_line                  = /home/CSCScience.ca/jtran/miniconda3/bin/staramr search --pointfinder-organism salmonella -o out GCF_001478105.1.fasta GCF_001931595.1.fasta
version                       = 1.0.0.dev0
start_time                    = 2019-03-26 15:38:10
end_time                      = 2019-03-26 15:38:12
total_minutes                 = 0.04
resfinder_db_dir              = /home/CSCScience.ca/jtran/Projects/staramr/staramr/databases/data/update/resfinder
resfinder_db_url              = https://bitbucket.org/genomicepidemiology/resfinder_db.git
resfinder_db_commit           = e8f1eb2585cd9610c4034a54ce7fc4f93aa95535
resfinder_db_date             = Mon, 16 Jul 2018 16:58
pointfinder_db_dir            = /home/CSCScience.ca/jtran/Projects/staramr/staramr/databases/data/update/pointfinder
pointfinder_db_url            = https://bitbucket.org/genomicepidemiology/pointfinder_db.git
pointfinder_db_commit         = 8706a6363bb29e47e0e398c53043b037c24b99a7
pointfinder_db_date       

This shows the command-line options used to run `staramr`, runtime, as well as the **ResFinder**,**PointFinder**, and **PlasmidFinder** database versions.

In [21]:
ls out/hits

plasmidfinder_GCF_001478105.1.fasta  resfinder_GCF_001478105.1.fasta
pointfinder_GCF_001931595.1.fasta    resfinder_GCF_001931595.1.fasta


This directory contains all the BLAST hits that were found in the `out/{resfinder,pointfinder,plasmidfinder}.tsv` files, in FASTA format.

In [22]:
head out/hits/resfinder_GCF_001931595.1.fasta

>aph(3')-Ia_7_X62115 isolate: GCF_001931595.1, contig: ref|NZ_CP016411.1|, contig_start: 300747, contig_end: 301562, resistance_gene_start: 1, resistance_gene_end: 816, hsp/length: 816/816, pid: 100.00%, plength: 100.00%
ATGAGCCATATTCAACGGGAAACGTCTTGCTCGAGGCCGCGATTAAATTCCAACCTGGAT
GCTGATTTATATGGGTATAGATGGGCTCGCGATAATGTCGGGCAATCAGGTGCGACAATC
TATCGATTGTATGGGAAGCCCAATGCGCCAGAGTTGTTTCTGAAACATGGCAAAGGTAGC
GTTGCCAATGATGTTACAGATGAGATGGTCAGACTAAACTGGCTGACGGCATTTATGCCT
CTTCCGACCATCAAGCATTTTATCCGTACTCCTGATGATGCATGGTTACTCACCACTGCG
ATCCCCGGGAAAACAGCATTCCAGGTATTAGAAGAATATCCTGATTCAGGTGAAAATATT
GTTGATGCGCTGGCAGTGTTCCTGCGCCGGTTGCATTCGATTCCTGTTTGTAATTGTCCT
TTTAACAGCGATCGCGTATTTCGTCTCGCTCAGGCGCAATCACGAATGAATAACGGTTTG
GTTGATGCTAGTGATTTTGATGACGAGCGTAATGGCTGGCCTGTTGAACAAGTCTGGAAA


## Step 4: Validation

Let's look back at our **summary.tsv** file.

In [23]:
# Show only Genotype
cut -f 1,2 out/summary.tsv | column -s$'\t' -t

Isolate ID       Genotype
GCF_001478105.1  blaCMY-2
GCF_001931595.1  aac(3)-IVa, aph(3')-Ia, aph(4)-Ia, blaCTX-M-65, dfrA14, floR, gyrA (D87Y), sul1, tet(A)


In [24]:
# Show only Predicted Phenotype
cut -f 1,3 out/summary.tsv | column -s$'\t' -t

Isolate ID       Predicted Phenotype
GCF_001478105.1  ampicillin, amoxicillin/clavulanic acid, cefoxitin, ceftriaxone
GCF_001931595.1  gentamicin, kanamycin, hygromicin, ampicillin, ceftriaxone, trimethoprim, chloramphenicol, ciprofloxacin I/R, nalidixic acid, sulfisoxazole, tetracycline


We can validate these results by comparing them to this drugs and AMR resistances available from NCBI.  Let's take a look:

### GCF_001478105.1

#### Genotypes

For **GCF_001478105.1** we can find the detected AMR genes by NCBI at <https://www.ncbi.nlm.nih.gov/pathogens/isolates/#/search/GCA_001478105.1>.  From here we see that `blaCMY-2` is listed under the **AMR geneotypes** column, which exactly matches what we see from `staramr`.

#### Predicted Phenotypes

The phenotypes are also in this same table under **AST Phenotypes** (or at <https://www.ncbi.nlm.nih.gov/biosample/SAMN02699230/>). This contains the list: `amoxicillin-clavulanic acid, ampicillin, cefoxitin, ceftiofur, ceftriaxone`. Comparing to the results from `staramr` we can see that `staramr` is missing `ceftiofur`.

### GCF_001931595.1

#### Genotypes

For **GCA_001931595.1** we can find the detected AMR genes by NCBI at <https://www.ncbi.nlm.nih.gov/pathogens/isolates/#/search/GCA_001931595.1>.  From here we see that `aac(3)-IV, aph(3')-Ia, aph(4)-Ia, blaCTX-M-65, dfrA14, floR, sul1, tet(A)` are listed under the **AMR geneotypes** column. When compared to `staramr`, it looks like `staramr` has one additional gene, mainly `gyrA (D87Y)`, which is a point mutation resistance. Also in `staramr` we have `aac(3)-IVa` instead of `aac(3)-IV`.

#### Predicted Phenotypes

The phenotypes are also in this same table under **AST Phenotypes** (or at <https://www.ncbi.nlm.nih.gov/biosample/SAMN03988471>). This contains the list (when including the *Intermediate* category): `ampicillin, ceftiofur, ceftriaxone, chloramphenicol, nalidixic acid, tetracycline, ciprofloxacin, gentamicin`. Comparing to the results from `staramr` we can see that `staramr` is missing `ceftiofur`, and `staramr` additionally includes `kanamycin, hygromicin, trimethoprim, sulfisoxazole`.

Overall, `staramr` matches pretty closely to what is found on NCBI for these genomes, but the differences observed here highlights the need for additional testing and development of the software and databases, which is an ongoing effort.